<a href="https://colab.research.google.com/github/mohammad2682/FVS-SVM-RKHS/blob/main/SVM_RKHS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn import svm

In [2]:
!git clone https://github.com/mohammad2682/FVS-SVM-RKHS

Cloning into 'FVS-SVM-RKHS'...
remote: Enumerating objects: 71, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 71 (delta 26), reused 41 (delta 17), pack-reused 0
Unpacking objects: 100% (71/71), done.


In [3]:
def sigma(df):
  N = df.shape[0]
  sigma = []
  for i in range(N):
    for j in range(i+1, N):
      x_i = df.iloc[[i], :-1]
      x_j = df.iloc[[j], :-1]
      sigma.append(np.linalg.norm(x_i.values - x_j.values))
  return 0.1 * np.max(sigma)

In [4]:
def K(FV1, FV2, sigma):
  dist = np.linalg.norm(FV1-FV2)
  return np.exp(-1*np.square(dist)/(2*sigma))

In [5]:
def give_KSS(S, sigma):
  KSS = []
  for vec1 in S:
    for vec2 in S:
      k = K(np.array(vec1), np.array(vec2), sigma = sigma)
      KSS.append(k)

  return np.array(KSS).reshape((len(S), len(S)))

In [6]:
def give_KSx(S, x, sigma):
  KSx = []
  for s in S:
    kx = K(np.array(s), x, sigma = sigma)
    KSx.append(kx)
  return np.array(KSx).reshape((len(S), 1))

In [7]:
df = pd.read_csv("FVS-SVM-RKHS/datasets/pima.csv")
df.head()

,preg,plas,pres,skin,insu,mass,pedi,age,class
0,14,175,62,30,0,33.6,0.212,38,tested_positive
1,4,146,78,0,0,38.5,0.520,67,tested_positive
2,15,136,70,32,110,37.1,0.153,43,tested_positive
3,3,107,62,13,48,22.9,0.678,23,tested_positive
4,3,169,74,19,125,29.9,0.268,31,tested_positive


In [8]:
df_test1 = df.iloc[:3, :]
df_test2 = df.iloc[-7:, :]
df_test = pd.concat([df_test1, df_test2]).reset_index(drop=True)
df_test = df_test[["preg", "plas", "class"]]

In [9]:
df_test

,preg,plas,class
0,14,175,tested_positive
1,4,146,tested_positive
2,15,136,tested_positive
3,6,114,tested_negative
4,6,92,tested_negative
5,5,117,tested_negative
6,4,83,tested_negative
7,7,119,tested_negative
8,1,95,tested_negative
9,1,181,tested_positive


In [10]:
sig = sigma(df_test)

In [11]:
sig

9.804590761474953

In [13]:
FV1 = df_test.iloc[0, :-1].values
tau = 0.1
S = [list(FV1)]
for vec in df_test.iloc[1:, :-1].values:
  KSx = give_KSx(S, vec, sigma = sig)
  KSS = give_KSS(S, sigma = sig)
  LF = abs(1-(np.dot(np.dot(np.transpose(np.array(KSx)), np.linalg.inv(KSS)), KSx)))
  print(LF)
  if LF > tau:
    S.append(list(vec))

[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[0.63937996]]
[[0.99982824]]
[[0.5297869]]
[[0.96880966]]
[[1.]]


In [15]:
KSx

array([[2.88244546e-005],
       [4.67709897e-028],
       [6.46230288e-050],
       [1.06207240e-100],
       [1.03731498e-176],
       [8.50064490e-092],
       [1.24743162e-213],
       [1.16862039e-086],
       [1.57412571e-164]])

In [16]:
KSS

array([[1.00000000e+000, 1.44276655e-021, 1.95656639e-034,
        1.48486377e-084, 1.01949463e-154, 5.03313914e-077,
        2.13182438e-190, 2.88529827e-071, 3.25857386e-146],
       [1.44276655e-021, 1.00000000e+000, 1.27467946e-005,
        1.70750887e-023, 2.13442072e-065, 2.24801519e-019,
        1.24895086e-088, 4.52001579e-017, 1.56677686e-058],
       [1.95656639e-034, 1.27467946e-005, 1.00000000e+000,
        3.06661062e-013, 2.13048609e-045, 6.16594075e-011,
        1.28175482e-065, 1.52030277e-008, 2.68614629e-042],
       [1.48486377e-084, 1.70750887e-023, 3.06661062e-013,
        1.00000000e+000, 1.90812442e-011, 6.00516482e-001,
        4.24283262e-022, 2.65561322e-001, 2.82528563e-009],
       [1.01949463e-154, 2.13442072e-065, 2.13048609e-045,
        1.90812442e-011, 1.00000000e+000, 1.36666074e-014,
        1.31057512e-002, 6.79704284e-017, 1.76597588e-001],
       [5.03313914e-077, 2.24801519e-019, 6.16594075e-011,
        6.00516482e-001, 1.36666074e-014, 1.0000000

In [14]:
list(FV1)

[14, 175]

In [ ]:
df['class'].unique()

array(['tested_positive', 'tested_negative'], dtype=object)

In [ ]:
df.groupby("class").agg("count")

,preg,plas,pres,skin,insu,mass,pedi,age
class,,,,,,,,
tested_negative,500,500,500,500,500,500,500,500
tested_positive,268,268,268,268,268,268,268,268
